# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

#### Install required packages

In [ ]:
# %pip install docarray tiktoken pypdf langchain openai python-dotenv

In [30]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [31]:
import os 

import openai

openai.api_key = os.environ['OPENAI_API_KEY']

In [32]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch

#### Load the document

In [33]:
file = 'sample.pdf'
loader = PyPDFLoader(file_path=file)

In [34]:
loader

#### First way

In [35]:
from langchain.indexes import VectorstoreIndexCreator

In [36]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [38]:
query = 'what are the seven spiritual laws success..?'
response = index.query(query)

In [39]:
print(response)

 The seven spiritual laws of success are the Law of Pure Potentiality, the Law of Giving, the Law of Karma, the Law of Least Effort, the Law of Intention and Desire, the Law of Detachment, and the Law of Dharma.


#### Second way

In [40]:
docs = loader.load()

In [41]:
len(docs)

39

In [42]:
docs[0]

Document(page_content='Deepak Chopra\nThe Seven\nspiritual\nlaws\nof\nSuccess', metadata={'source': 'sample.pdf', 'page': 0})

In [43]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [44]:
embed = embeddings.embed_query("what are the seven spiritual laws success..?")

In [45]:
print(len(embed))

1536


In [46]:
print(embed[:5])

[0.015392657369375229, 0.00533225666731596, 0.002493275096639991, -0.023815317079424858, -0.01982049085199833]


In [47]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [49]:
query = 'what are the seven spiritual laws success..?'

In [50]:
docs = db.similarity_search(query)

In [52]:
len(docs)

4

In [53]:
docs[0]

Document(page_content='Deepak Chopra\nThe Seven\nspiritual\nlaws\nof\nSuccess', metadata={'source': 'sample.pdf', 'page': 0})

In [56]:
llm = ChatOpenAI(temperature = 0.0)

In [57]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [59]:
qdocs

'Deepak Chopra\nThe Seven\nspiritual\nlaws\nof\nSuccessINTRODUCTION\nAlthough this book is titled The Seven Spiritual Laws of Success,  it could also be called The\nSeven Spiritual Laws of Life,  because these are the same principles that nature uses to create\neverything in material existence \x97 everything we can see, hear, smell, taste, or touch.\nIn my book, Creating Affluence: Wealth Consciousness in the Field of All Possibilities,  I\nhave outlined the steps to wealth consciousness based on a true understanding of the workingsof nature. The Seven Spiritual Laws of  Success form the essence of this teaching. When this\nknowledge is incorporated in your consciousness, it will give you the ability to create unlimited\nwealth with effortless ease, and to experience success in every endeavour.\nSuccess in life could be defined as the continued expansion of happiness and the progres-\nsive realization of worthy goals. Success is the ability to fulfill your desires with effortless ease

In [60]:
response = llm.call_as_llm(f'{qdocs} what are the seven spiritual laws success..?') 

In [61]:
print(response)

1. The Law of Pure Potentiality: This law states that our essential nature is pure consciousness, and we have unlimited potential to create anything we desire. To access this potential, we must practice silence, meditation, and non-judgment.

2. The Law of Giving: This law states that the universe operates through dynamic exchange, and the more we give, the more we receive. Giving can take many forms, including time, money, and love.

3. The Law of Karma: This law states that every action we take has a consequence, and we must take responsibility for our actions. By choosing actions that bring happiness and success to others, we create positive karma for ourselves.

4. The Law of Least Effort: This law states that we can achieve our goals with minimal effort by aligning ourselves with the natural flow of the universe. This means accepting things as they are, taking responsibility for our situation, and not forcing things to happen.

5. The Law of Intention and Desire: This law states t

Third way

In [62]:
retriever = db.as_retriever()

In [63]:
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7ff854e3b3d0>, search_type='similarity', search_kwargs={})

In [64]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [65]:
query = "what are the seven spiritual laws success..?"

In [66]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [67]:
print(response)

The Seven Spiritual Laws of Success are:

1. The Law of Pure Potentiality
2. The Law of Giving
3. The Law of Karma or Cause and Effect
4. The Law of Least Effort
5. The Law of Intention and Desire
6. The Law of Detachment
7. The Law of Dharma or Purpose in Life


#### First way extention

In [68]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [69]:
index

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7ff85686f1c0>)

In [70]:
query = 'what are the seven spiritual laws success..?'
response = index.query(query)

In [71]:
print(response)

 The seven spiritual laws of success are the Law of Pure Potentiality, the Law of Giving, the Law of Karma, the Law of Least Effort, the Law of Intention and Desire, the Law of Detachment, and the Law of Dharma.
